In [209]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import time

In [29]:
import pickle
import numpy as np
import time
import pandas as pd
from nltk.tokenize import word_tokenize
from unidecode import unidecode

In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 5732083.41B/s]


In [81]:
#df_posts = pickle.load( open( "df_posts_p360.p", "rb" ) )
all_docs = pickle.load( open("all_docs.p", "rb" ) )

In [98]:
def findall(p, s):
    '''Yields all the positions of
    the pattern p in the string s.'''
    i = s.find(p)
    while i != -1:
        yield i
        i = s.find(p, i+1)

In [222]:
def pre_process(paragraph):
    # convert input corpus to lower case.   
    corpus = paragraph.lower()
    # collecting a list of stop words from nltk and punctuation form
    corpus = corpus.replace('.','. [SEP]').replace('?','? [SEP]').replace('!','! [SEP]')
    corpus = corpus.replace('(','[SEP] (').replace(')',') [SEP]')
    corpus = unidecode(corpus)

    tok = 'Click to expand'
    if tok in corpus:
        index = corpus.find(tok)
        corpus = corpus[index+len(tok)+1:]
    
    for i in range(2,10):
        corpus = corpus.replace(i*' ', ' ')
        
    if corpus[-5:] != '[SEP]':
        corpus += ' [SEP]'

    #print('----')
    #print(corpus)
    #corpus = '[CLS] '+ corpus
    
    #replace '[SEP]' with '[CLS]' every other time.
    result = []
    split_corp  = corpus.split('[SEP]')
    split_corp2 = [ split_corp[i:i+2] for i in range(0, len(split_corp), 2) ]
    for i, sc in enumerate(split_corp2):
        sent = '[SEP]'.join(sc)
        if sent[:5] !=['[CLS]']:
            sent = '[CLS] ' +sent if i==0 else '[CLS]' +sent
        if sent[-5:] != '[SEP]':
            sent += ' [SEP]'
        sent = sent.replace('  ',' ')
        if sent != '[CLS] [SEP]':
            result.append( sent )
    corpus = result[:]
    
    #print(corpus)
    #print('XXXXX')
    tokenized_text = [tokenizer.tokenize(c) for c in corpus]
    seg_ids = [(tt.index('[SEP]')+1)*[0] + (len(tt)-tt.index('[SEP]')-1)*[1] for tt in tokenized_text]
    indexed_tokens = [tokenizer.convert_tokens_to_ids(tt) for tt in tokenized_text]
    
    lens = [len(tt) for tt in tokenized_text]
    if max(lens)>500:
        print(corpus)
    
    '''if True:
        print(corpus)
        #print('before')
        tokenized_text = tokenizer.tokenize(corpus)
        #print('mid', len(corpus))
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        #print('after')
        print('-----', len(corpus))
        print(len(tokenized_text), len(indexed_tokens))
        #tokenized_text, indexed_tokens = [], []
    
    tokenized_text = tokenizer.tokenize(corpus)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)'''
    
    return tokenized_text, indexed_tokens, seg_ids

In [222]:
def pre_process0(paragraph):
    # convert input corpus to lower case.   
    corpus = paragraph.lower()
    # collecting a list of stop words from nltk and punctuation form
    corpus = corpus.replace('.','. [SEP]').replace('?','? [SEP]').replace('!','! [SEP]')
    corpus = corpus.replace('(','[SEP] (').replace(')',') [SEP]')
    corpus = unidecode(corpus)

    tok = 'Click to expand'
    if tok in corpus:
        index = corpus.find(tok)
        corpus = corpus[index+len(tok)+1:]
    
    for i in range(2,10):
        corpus = corpus.replace(i*' ', ' ')
        
    if corpus[-5:] != '[SEP]':
        corpus += ' [SEP]'

    #replace '[SEP]' with '[CLS]' every other time.
    result = []
    split_corp  = corpus.split('[SEP]')
    split_corp2 = [ split_corp[i:i+2] for i in range(0, len(split_corp), 2) ]
    for i, sc in enumerate(split_corp2):
        sent = '[SEP]'.join(sc)
        if sent[:5] !=['[CLS]']:
            sent = '[CLS] ' +sent if i==0 else '[CLS]' +sent
        if sent[-5:] != '[SEP]':
            sent += ' [SEP]'
        sent = sent.replace('  ',' ')
        if sent != '[CLS] [SEP]':
            result.append( sent )
    corpus = result[:]
 
    tokenized_text = [tokenizer.tokenize(c) for c in corpus]
    seg_ids = [(tt.index('[SEP]')+1)*[0] + (len(tt)-tt.index('[SEP]')-1)*[1] for tt in tokenized_text]
    indexed_tokens = [tokenizer.convert_tokens_to_ids(tt) for tt in tokenized_text]
    
    lens = [len(tt) for tt in tokenized_text]
    if max(lens)>500:
        print(corpus)
    
    return tokenized_text, indexed_tokens, seg_ids

In [326]:
def padding(indx_list, m):
    ##indx_list: [[]]
    padded_indx = [indx+[0]*(m-len(indx)) if len(indx)<=m else indx[:m-1] + indx[-1:] for indx in indx_list]
    
    return padded_indx

In [349]:
t0 = time.time()
m=128
all_tokenized_text, all_indexed_tokens, all_seg_ids = [],[],[]
grouped_tokenized_text, grouped_indexed_tokens, grouped_seg_ids = [],[],[]
for i, doc in enumerate(all_docs[5:10]):
    post_tokenized_text, post_indexed_tokens, post_seg_ids = [], [], []
    for post in doc[:-2]: 
        tokenized_text, indexed_tokens, seg_ids = pre_process(post)
        all_tokenized_text += tokenized_text
        all_indexed_tokens += indexed_tokens
        all_seg_ids += seg_ids
        post_tokenized_text.append(tokenized_text)
        post_indexed_tokens.append(padding(indexed_tokens, m))
        post_seg_ids.append(padding(seg_ids, m) )
    grouped_tokenized_text.append(post_tokenized_text + doc[-2:])
    grouped_indexed_tokens.append(post_indexed_tokens + doc[-2:])
    grouped_seg_ids.append(post_seg_ids + doc[-2:]) 

#print(grouped_indexed_tokens[4][:])

In [351]:
print(len(all_tokenized_text), len(all_indexed_tokens), len(all_seg_ids))
print(len(grouped_tokenized_text), len(grouped_indexed_tokens), len(grouped_seg_ids))

168 168 168
5 5 5


In [341]:
#padding the tokenized indexed tokens and seg_ids FOR training
all_tokenized_text2, all_indexed_tokens2, all_seg_ids2 = [],[],[]
for indx, text, seg in zip(all_indexed_tokens, all_tokenized_text, all_seg_ids):
    if len(indx)>m:
        all_tokenized_text2.append(text)
        all_indexed_tokens2.append(indx[:m-1] + indx[-1:])
        all_seg_ids2.append(seg[:m-1] + seg[-1:])
    else:
        all_tokenized_text2.append(text)
        all_indexed_tokens2.append(indx+[0]*(m-len(indx)))
        all_seg_ids2.append(seg+[0]*(m-len(indx)))

In [342]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor(all_indexed_tokens2)
segments_tensors = torch.tensor(all_seg_ids2)

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [285]:
all_tokenized_post, all_indexed_post, all_seg_post = [],[],[]
all_title_url = []
k=0
for i, doc in enumerate(all_docs):
    num_post = len(doc[:-2])
    title_url = doc[-2:]
    all_tokenized_post.append( all_tokenized_text2[k:(num_post+k)] + title_url ) 
    all_indexed_post.append( all_indexed_tokens2[k:(num_post+k)] + title_url ) # ..token2 means it's padded
    all_seg_post.append( all_seg_ids2[k:(num_post+k)] + title_url )
    k += num_post

print(k, len(all_indexed_tokens2))
    

107825 349533


In [343]:
search = 'rotor replacement'
tokenized_search, indexed_search, seg_search_ids = pre_process(search)
print(len(indexed_search), len(seg_search_ids))

# padding the input to make it same dimension
indexed_search = padding(indexed_search, m)
seg_search_ids = padding(seg_search_ids, m)
print(len(indexed_search[0]), len(seg_search_ids[0]))

tokens_search_tensor = torch.tensor(indexed_search)
seg_search_tensor = torch.tensor(seg_search_ids)

print(indexed_search[0])

1 1
128 128
[101, 18929, 6110, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [274]:
# https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_search_tensor, seg_search_tensor)
    
token_embeddings = torch.stack(encoded_layers, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

'''search_vec_sum = []
for token in token_embeddings:
    sum_vec = torch.sum(token[-4:], dim=0)
    search_vec_sum.append(sum_vec)

search_vec = torch.mean(search_vec_sum, dim=0)
print(len(search_vec))
#print(len(search_vec_sum), len(search_vec_sum[0]))'''

'search_vec_sum = []\nfor token in token_embeddings:\n    sum_vec = torch.sum(token[-4:], dim=0)\n    search_vec_sum.append(sum_vec)\n\nsearch_vec = torch.mean(search_vec_sum, dim=0)\nprint(len(search_vec))\n#print(len(search_vec_sum), len(search_vec_sum[0]))'

In [301]:
# `token_vecs` is a tensor with shape [22 x 768]
search_token_vecs = encoded_layers[11][0]
search_emb = torch.mean(search_token_vecs, dim=0)
print(np.shape(np.array(search_emb)))

(768,)


In [352]:
doc_emb = []
for indexed_tokens, seg_ids in zip(grouped_indexed_tokens, grouped_seg_ids):
    temp = 0
    for indx, ids in zip(indexed_tokens[:-2], seg_ids[:-2]):
        #print(len(ids),len(indx))
        index_tensor = torch.tensor(indx)
        seg_tensor = torch.tensor(ids)
        with torch.no_grad():
            encoded_layers, _ = model(index_tensor, seg_tensor)
        token_vecs = encoded_layers[11][0]
        emb = torch.mean(token_vecs, dim=0)
        temp += np.array(emb)
    
    post_emb = temp/len(indexed_tokens[:-2])
    doc_emb.append([seg_ids[-2], seg_ids[-1], post_emb])


13 13
3 3
3 3
2 2
1 1
3 3
3 3
2 2
2 2
4 4
4 4
9 9
3 3
4 4
2 2
1 1
3 3
3 3
5 5
2 2
1 1
2 2
2 2
5 5
4 4
2 2
4 4
1 1
2 2
6 6
2 2
1 1
2 2
4 4
4 4
3 3
2 2
2 2
4 4
3 3
1 1
1 1
4 4
5 5
2 2
2 2
5 5
5 5
1 1
1 1
1 1
2 2
4 4
1 1
5 5
